In [1]:
import matplotlib as plt
import pandas as pd
import requests
import seaborn
import glob
import statsmodels.api as sm
from datetime import date
import numpy as np

In [2]:
def cointegration_test_ct(S1, S2,date1,date2):
    i = S1.index.intersection(S2.index)
    length = len(i) -1
    print(i[0])
    print(i[length])
    S11=S1[i[0]:i[length]]
    S22=S2[i[0]:i[length]]
    x = (S11[date1:date2]['Adjclose'])
    y = (S22[date1:date2]['Adjclose'])
    if len(x) == 0 or len(y) == []:
        return [[0,0,0]]
    # Step 1: regress on variable on the other
    ols_result = sm.OLS(y, x).fit()
    return sm.tsa.stattools.adfuller(ols_result.resid,regression  = 'ct'),ols_result.resid

In [3]:
def cointegration_test(S1, S2,date1,date2):
    i = S1.index.intersection(S2.index)
    length = len(i) -1
    S11=S1[i[0]:i[length]]
    S22=S2[i[0]:i[length]]
    x = (S11[date1:date2]['Adjclose'])
    y = (S22[date1:date2]['Adjclose'])
    if len(x) == 0 or len(y) == []:
        return [[0,0,0]]
    # Step 1: regress on variable on the other
    ols_result = sm.OLS(y, x).fit()
    return sm.tsa.stattools.adfuller(ols_result.resid),ols_result.resid

In [4]:
pairs = []
df = pd.read_excel('pairs.xlsx')
for i in range(204):
    first = '2013/' + str(df['first'][i]) + '.csv'
    second = '2013/' + str(df['second'][i]) + '.csv'
    pairs.append((first,second))

In [5]:
def trade(S1, S2, window1, window2):
    
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    
    # Compute rolling mean and rolling standard deviation
    ratios = S1/S2
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()
    std = ratios.rolling(window=window2,
                        center=False).std()
    zscore = (ma1 - ma2)/std
    
    #zscore = zscore[]
    # Simulate trading
    # Start with no money and no positions
    money = 10000
    returns = []
    countS1 = 0
    countS2 = 0
    for i in zscore.index:
        # Sell short if the z-score is > 1
        if zscore[i] > 1 and (countS1==0) and (countS2==0) :
            countS1 = -(money/2)/S1[i]
            countS2 = (money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            #print('Selling Ratio %s %s %s %s'%(money, ratios[i], countS1,countS2))
            #print('Selling Ratio %s %s %s %s %s'%(money, ratios[i], countS1,countS2,zscore[i]))
        # Buy long if the z-score is < 1
        elif zscore[i] < -1 and (countS1==0) and (countS2==0):
            countS1 = (money/2)/S1[i]
            countS2 = -(money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            #print('Buying Ratio %s %s %s %s'%(money,ratios[i], countS1,countS2))
            #print('Buying Ratio %s %s %s %s %s'%(money,ratios[i], countS1,countS2,zscore[i]))
        # Clear positions if the z-score between -.5 and .5
        #elif abs(zscore[i]) < 1:
        elif abs(zscore[i]) < 0.5:
            money += S1[i] * countS1 + S2[i] * countS2
            countS1 = 0
            countS2 = 0
            #print('Exit pos %s %s %s %s'%(money,ratios[i], countS1,countS2))
            #print(S1[i])
            #print(S2[i])
            #returns.append(money)
            
            
            #print('Exit pos %s %s %s %s %s'%(money,ratios[i], countS1,countS2,zscore[i]))
            
            
    return money

In [6]:
def trade1(data1, data2,date1,date2,window):
    S1 = data1[date1:date2]['Adjclose']
    S2 = data2[date1:date2]['Adjclose']
    window1 = window[0] + 3
    window2 = window[1] + 35
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0

    # Compute rolling mean and rolling standard deviation
    ratios = S1/S2
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()
    std = ratios.rolling(window=window2,
                        center=False).std()
    zscore = (ma1 - ma2)/std
    
   
    money = 10000
    returns = [10000]
    countS1 = 0
    countS2 = 0
    for i in zscore.index:
        # Sell short if the z-score is > 1
        if zscore[i] > 1 and (countS1==0) and (countS2==0) :
            countS1 = -(money/2)/S1[i]
            countS2 = (money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            print(i)
            print('Selling Ratio %s %s %s %s'%(money, ratios[i], countS1,countS2))
            #print('Selling Ratio %s %s %s %s %s'%(money, ratios[i], countS1,countS2,zscore[i]))
        # Buy long if the z-score is < 1
        elif zscore[i] < -1 and (countS1==0) and (countS2==0):
            countS1 = (money/2)/S1[i]
            countS2 = -(money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            print(i)
            print('Buying Ratio %s %s %s %s'%(money,ratios[i], countS1,countS2))
        # Clear positions if the z-score between -.5 and .5
        #elif abs(zscore[i]) < 1:
        elif abs(zscore[i]) < 0.5:
            money += S1[i] * countS1 + S2[i] * countS2
            countS1 = 0
            countS2 = 0
            print(i)
            #print(S1[i])
            #print(S2[i])
            print('Exit pos %s %s %s %s'%(money,ratios[i], countS1,countS2))
            returns.append(money)
            
    print('test' + str(returns[-1]))
    #testing.append(returns[-1])
    #return pd.Series(returns)
    return returns[-1]

In [7]:
def avg_optimal(S1,S2,date1,date2):
    main = []
    set1 = (S1[date1:date2])
    set2 = (S2[date1:date2])
    for f in range(3,25):
        row =  [trade(set1['Adjclose'],set2['Adjclose'],f,s) for s in range(35,200)] 
        main.append(row)
    scores = np.array(main)
    row_avg = np.mean(scores,axis=1)
    col_avg = np.mean(scores,axis=0)
    fma = np.argmax(row_avg) + 3
    sma = np.argmax(col_avg) + 35
    print(fma)
    print(sma)
    return (fma,sma)

In [8]:
periods = [("20080701","20130630")]
test = [('20130630','20140630')]

In [9]:
testing = []

for item in pairs:
    print(item)
    name1 = item[0]
    name2 = item[1]
    data1 = (pd.read_csv(name1,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    money = 0
    train_p = periods[0]
    test_p = test[0]
    print(train_p)
    window = avg_optimal(data1,data2,train_p[0],train_p[1])
    print(test_p)
    temp = trade1(data1,data2,test_p[0],test_p[1],window)
    print("Return for" + test_p[0]+test_p[1] + "is" + str(temp))
    testing.append(temp)
    money += temp

('2013/CENTURYTEX.BO.csv', '2013/GRASIM.BO.csv')
('20080701', '20130630')


KeyboardInterrupt: 

In [ ]:
arr = []
arr_ct = []

for item in pairs:
    print(item)
    name1 = item[0]
    name2 = item[1]
    data1 = (pd.read_csv(name1,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    
    money = 0
    returns = []
    train_p = periods[0]
    test_p = test[0]
    value1 = cointegration_test(data1,data2,train_p[0],train_p[1])[0][1]
    value2 = cointegration_test_ct(data1,data2,train_p[0],train_p[1])[0][1]
    print(train_p)
    print(str(value1) +  "  " + str(value2))
    print("---")
    arr.append(value1)
    arr_ct.append(value2)